In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Aux functions

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [3]:
# # convert series to supervised learning
# def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
# 	"""
# 	Frame a time series as a supervised learning dataset.
# 	Arguments:
# 		data: Sequence of observations as a list or NumPy array.
# 		n_in: Number of lag observations as input (X).
# 		n_out: Number of observations as output (y).
# 		dropnan: Boolean whether or not to drop rows with NaN values.
# 	Returns:
# 		Pandas DataFrame of series framed for supervised learning.
# 	"""
# 	n_vars = 1 if type(data) is list else data.shape[1]
# 	df = pd.DataFrame(data)
# 	cols, names = list(), list()
# 	# input sequence (t-n, ... t-1)
# 	for i in range(n_in, 0, -1):
# 		cols.append(df.shift(i))
# 		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
# 	# forecast sequence (t, t+1, ... t+n)
# 	for i in range(0, n_out):
# 		cols.append(df.shift(-i))
# 		if i == 0:
# 			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
# 		else:
# 			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
# 	# put it all together
# 	agg = pd.concat(cols, axis=1)
# 	agg.columns = names
# 	# drop rows with NaN values
# 	if dropnan:
# 		agg.dropna(inplace=True)
# 	return agg

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

## Dataset Energy Appliance

In [5]:
filename = '/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/energydata_complete.csv'
data = pd.read_csv(filename)
data.pop('date')
data.pop('rv1')
data.pop('rv2')
data.dropna(inplace=True)
data.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [6]:
# values = data.values
# values

### Dataset Energy Appliance: columns

In [7]:
# cols = data.columns
# list_cols = list(cols)
# list_cols.append('Appliances_t+1')
# print(list_cols)

### Dataset Energy Appliance: Preprocessing, Lag = 1, t+1

In [8]:
# # frame as supervised learning
# reframed = series_to_supervised(values, 1, 1)
# # drop columns we don't want to predict
# reframed.drop(reframed.columns[[27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
# #reframed.drop(reframed.columns[[0,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
# reframed.head()

In [9]:
# data = pd.DataFrame(reframed.values,columns=list_cols)
data = clean_dataset(data)
data

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60.0,30.0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000
1,60.0,30.0,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000
2,50.0,30.0,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000
3,50.0,40.0,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000
4,60.0,40.0,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,100.0,0.0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333
19731,90.0,0.0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000
19732,270.0,10.0,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667
19733,420.0,10.0,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,...,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333


## Model: PcaWeightedMVFTS - PCA-MVFTS

In [10]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
                       endogen_variable = 'Appliances',
                       n_part = 50)

In [14]:
def sliding_window_pca(data,n_windows,train_size,dim,fs):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": [],
         "mae": []
    }

    order = 1
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:

            pca_wmvfts = PcaWeightedMVFTS(n_components = dim,
                       endogen_variable = 'Appliances',
                       n_part = fs)
            
#             data_train = ttrain.loc[:,'Appliances':'Tdewpoint']
#             data_test = ttest.loc[:,'Appliances':'Tdewpoint']
            
            model, pca_reduced_train = pca_wmvfts.run_train_model(ttrain)
            forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,ttest)

             #Teste e cálculo dos erros da previsão
            appliance_test = ttest['Appliances'].values
#             appliance_test = appliance_test[:len(appliance_test)-order]
#             forecast = forecast[order:]

            rmse = Measures.rmse(appliance_test,forecast)
            mape = Measures.mape(appliance_test,forecast)
            smape = Measures.smape(appliance_test,forecast)
            
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(appliance_test, forecast)

            result["rmse"].append(round(rmse,3))
            result["mape"].append(round(mape,3))
            result["smape"].append(round(smape,3))
            result["mae"].append(round(mae,3))
            result["window"].append(ct)
        
    measures = pd.DataFrame(result)
    return measures



In [15]:
dimensions = [2,3,4,5,6]
fuzzy_sets = [10,20,30,40,50]

final_result = {
    "dimensions": [],
    "fuzzy_sets": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "smape": []
}

n_windows = 30
train_size = 0.75

for dim in dimensions:
    for fs in fuzzy_sets:
        measures = sliding_window_pca(data,n_windows,train_size,dim,fs)
        final_result["dimensions"].append(dim)
        final_result["fuzzy_sets"].append(fs)
        
        rmse = round(statistics.mean(measures['rmse']),3)
        mape = round(statistics.mean(measures['mape']),3)
        smape = round(statistics.mean(measures['smape']),3)
        mae = round(statistics.mean(measures['mae']),3)
        
        final_result["rmse"].append(rmse)
        final_result["mape"].append(mape)
        final_result["smape"].append(smape)
        final_result["mae"].append(mae)
        
        print(f'Results: {(dim,fs,rmse,mae,mape,smape)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics PCA-WMVFTS (test): ")
final_measures_pca

Results: (2, 10, 28.378, 20.729, 29.656, 11.512)
Results: (2, 20, 17.643, 9.237, 11.797, 4.897)
Results: (2, 30, 11.561, 4.771, 5.65, 2.349)
Results: (2, 40, 8.686, 2.985, 3.172, 1.3)
Results: (2, 50, 4.868, 2.04, 1.75, 0.743)
Results: (3, 10, 18.052, 9.598, 13.841, 5.303)
Results: (3, 20, 5.136, 2.312, 2.221, 0.989)
Results: (3, 30, 2.428, 1.364, 0.691, 0.309)
Results: (3, 40, 1.246, 1.09, 0.276, 0.126)
Results: (3, 50, 0.705, 0.994, 0.136, 0.061)
Results: (4, 10, 8.849, 4.037, 5.347, 1.885)
Results: (4, 20, 1.468, 1.195, 0.481, 0.21)
Results: (4, 30, 0.432, 0.974, 0.113, 0.053)
Results: (4, 40, 0.147, 0.934, 0.064, 0.028)
Results: (4, 50, 0.088, 0.92, 0.036, 0.015)
Results: (5, 10, 3.336, 1.995, 1.875, 0.813)
Results: (5, 20, 0.322, 0.966, 0.115, 0.054)
Results: (5, 30, 0.087, 0.919, 0.034, 0.014)
Results: (5, 40, 0.045, 0.913, 0.025, 0.009)
Results: (5, 50, 0.039, 0.912, 0.024, 0.009)
Results: (6, 10, 1.753, 1.366, 0.871, 0.388)
Results: (6, 20, 0.113, 0.923, 0.035, 0.015)
Results: 

,dimensions,fuzzy_sets,rmse,mae,mape,smape
0,2,10,28.378,20.729,29.656,11.512
1,2,20,17.643,9.237,11.797,4.897
2,2,30,11.561,4.771,5.650,2.349
3,2,40,8.686,2.985,3.172,1.300
4,2,50,4.868,2.040,1.750,0.743
5,3,10,18.052,9.598,13.841,5.303
6,3,20,5.136,2.312,2.221,0.989
7,3,30,2.428,1.364,0.691,0.309
8,3,40,1.246,1.090,0.276,0.126
9,3,50,0.705,0.994,0.136,0.061


In [16]:
final_measures_pca

,dimensions,fuzzy_sets,rmse,mae,mape,smape
0,2,10,28.378,20.729,29.656,11.512
1,2,20,17.643,9.237,11.797,4.897
2,2,30,11.561,4.771,5.650,2.349
3,2,40,8.686,2.985,3.172,1.300
4,2,50,4.868,2.040,1.750,0.743
5,3,10,18.052,9.598,13.841,5.303
6,3,20,5.136,2.312,2.221,0.989
7,3,30,2.428,1.364,0.691,0.309
8,3,40,1.246,1.090,0.276,0.126
9,3,50,0.705,0.994,0.136,0.061


In [17]:
final_measures_pca.to_csv (r'pca_wmvfts_dim_fsets.csv', index = False, header=True)